In [6]:
sample_data_folder = "/Users/dipsy/github/effect/effect-data/isi/source-analysis/data"
import os
import json
source_files = os.listdir(sample_data_folder)

In [7]:
print (source_files)

['asu-hacking-items.jl', 'asu-hacking-posts.jl', 'asu-twitter.jl', 'asu-zero-day-products.jl', 'hackmageddon.jl', 'hg-abusech.jl', 'hg-blogs.jl', 'hg-conference.jl', 'hg-cpe.jl', 'hg-cve.jl', 'hg-msbulletin.jl', 'hg-taxii.jl', 'hg-zdi.jl', 'isi-twitter.jl']


In [113]:
i = 1
for filename in source_files:
    source_name = filename[0:len(filename)-3]
    print(str(i) + ":" + source_name)
    i += 1

1:asu-hacking-items
2:asu-hacking-posts
3:asu-twitter
4:asu-zero-day-products
5:hackmageddon
6:hg-abusech
7:hg-blogs
8:hg-conference
9:hg-cpe
10:hg-cve
11:hg-msbulletin
12:hg-taxii
13:hg-zdi
14:isi-twitter


In [99]:
def get_attributes_in_dict(dictionary):
    result = set()
    for (key, value) in dictionary.items():
        if key == "affected_products":
            result.add(key)
        elif type(value) == dict:
            inner_set = get_attributes_in_dict(value)
            for x in inner_set:
                inner_value = key + "." + x
                result.add(inner_value)
        elif type(value) == list or type(value) == set:
            for list_elem in value:
                if type(list_elem) == dict:
                    inner_set = get_attributes_in_dict(list_elem)
                    for x in inner_set:
                        inner_value = key + "." + x
                        result.add(inner_value)
        else:
            result.add(key)
    return result

In [100]:
def get_attributes_in_jsonlines(json_lines):
    attributes = set()
    for json_line in json_lines:
        attribs = get_attributes_in_dict(json_line)
        for a in attribs:
            attributes.add(a)
    atts_list = list(attributes)
    atts_list.sort()
    return atts_list

In [101]:
def get_attributes_in_file(filename):
    f = open(filename)
    attributes = set()
    for line in f:
        json_line = json.loads(line)
        attribs = get_attributes_in_dict(json_line)
        for a in attribs:
            attributes.add(a)
    atts_list = list(attributes)
    atts_list.sort()
    return atts_list


In [112]:
for filename in source_files:
    source_name = filename[0:len(filename)-3]
    print ("=====================================================")
    print(source_name)
    print ("-------------------------")
    attributes = get_attributes_in_file(sample_data_folder + "/" + filename)
    for attrib in attributes:
        print(attrib)

asu-hacking-items
-------------------------
autoTranslated
entity
entity.entityEnglishId
entity.entityId
entity.freebaseTypes
entity.matchedText
entity.type
financialTags
itemCategory
itemCve
itemDescription
itemEscrowinfo
itemId
itemLastViewDate
itemMs
itemName
itemQuantityLeft
itemQuantitySold
itemShippedFrom
itemShippedTo
itemSuccessfulTrans
itemVendorRating
itemViews
language
marketplaceId
postedDate
recordedDate
recordedTime
scrapedDate
sellingPrice.sellingPrice_Usd
sellingPriceUsd
softwareTags
tags
uid
asu-hacking-posts
-------------------------
autoTranslated
boardsName
entity
entity.entityEnglishId
entity.entityId
entity.freebaseTypes
entity.matchedText
entity.type
financialTags
forumsId
indicator
language
postContent
postCve
postMs
postedDate
postedTime
postsId
recordedDate
recordedTime
recorded_Time
scrapedDate
softwareTags
tags
topicId
topicsName
uid
asu-twitter
-------------------------
content_type
conversation_id
favorites
hashtags
mentions
recorded_time_seconds
retweet
t

In [103]:
import requests
from requests.auth import HTTPBasicAuth

def byteify(input):
    if isinstance(input, dict):
        return {byteify(key): byteify(value)
                for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [byteify(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

def download_api(url, username=None, password=None, headers=None, timeout=600):
    #timeout is in seconds
    auth = None
    if username is not None:
        auth = HTTPBasicAuth(username, password)
    print ("Download:" + url)
    response = requests.get(url, verify=False,	auth=auth, headers=headers, timeout=timeout)
    text = response.text
    if len(text) > 0:
        return (json.loads(text))
    return []

In [111]:
date = "2017-07-11"

from datetime import datetime


dt = datetime.strptime(date, "%Y-%m-%d")
epoch=datetime(1970,1,1)
td = dt - epoch
timestamp = int((td.microseconds + (td.seconds + td.days * 86400) * 10**6) / 10**6) * 1000

asu_date_filter = "from=" + date + "&to=" + date + "&start=0&limit=10"
hg_date_filter = date + "T00:00:00Z/pages/0?limit=10"
apis = {
    "asu-zero-day-products": "https://apigargoyle.com/GargoyleApi/getZerodayProducts?order=scrapedDate&" + asu_date_filter,
    "asu-hacking-items":  "https://apigargoyle.com/GargoyleApi/getHackingItems?order=scrapedDate&" + asu_date_filter,
    "asu-hacking-posts": "https://apigargoyle.com/GargoyleApi/getHackingPosts?order=scrapedDate&" + asu_date_filter,
    "asu-twitter": "https://apigargoyle.com/GargoyleApi/getTwitterData?" + asu_date_filter,
    "hg-blogs": "https://effect.hyperiongray.com/api/blogs/updates/" + hg_date_filter,
    "hg-abusech": "https://effect.hyperiongray.com/api/ransomware-tracker/updates/" + hg_date_filter,
    "hg-zdi": "https://effect.hyperiongray.com/api/zdi/?query={\"date\":{\"$gte\": {\"$date\": " + str(timestamp) + "}}}",
    "hg-cpe": "https://effect.hyperiongray.com/api/cpe/updates/" + hg_date_filter,
    "hg-cve": "https://effect.hyperiongray.com/api/cpe/updates/" + hg_date_filter,
    "hg-conference": "https://effect.hyperiongray.com/api/conferences/updates/" + hg_date_filter,
    "hg-taxii": "https://effect.hyperiongray.com/api/lehigh_edu/updates/" + hg_date_filter,
}

asu_headers = {"userId" :"usc","apiKey": "7a417415-d5f8-4883-82b6-b55c3a0e3d3a", "Connection" : "close"}
source_attribs = {}
for api_name in apis:
    api_url = apis[api_name]
    if api_name.find("asu") != -1:
        data = download_api(api_url, headers=asu_headers)
    else:
        data = download_api(api_url, username='isi', password='KSIDOOIWHJu8ewhui8923y8gYGuYGASYUHjksahuihIHU')
        
    if type(data) == dict:
        attributes = get_attributes_in_jsonlines(data['results'])
    else:
        attributes = get_attributes_in_jsonlines(data)
    source_attribs[api_name] = attributes
    
for source_name in source_attribs:
    attributes = source_attribs[source_name]
    print ("=====================================================")
    print(source_name)
    print ("-------------------------")
    print (attributes)

Download:https://apigargoyle.com/GargoyleApi/getTwitterData?from=2017-07-11&to=2017-07-11&start=0&limit=10


/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


Download:https://effect.hyperiongray.com/api/cpe/updates/2017-07-11T00:00:00Z/pages/0?limit=10


/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


Download:https://effect.hyperiongray.com/api/cpe/updates/2017-07-11T00:00:00Z/pages/0?limit=10


...

Download:https://apigargoyle.com/GargoyleApi/getZerodayProducts?order=scrapedDate&from=2017-07-11&to=2017-07-11&start=0&limit=10
Download:https://effect.hyperiongray.com/api/ransomware-tracker/updates/2017-07-11T00:00:00Z/pages/0?limit=10


/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


Download:https://effect.hyperiongray.com/api/zdi/?query={"date":{"$gte": {"$date": 1499731200000}}}
Download:https://apigargoyle.com/GargoyleApi/getHackingPosts?order=scrapedDate&from=2017-07-11&to=2017-07-11&start=0&limit=10


/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/Users/dipsy/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


asu-twitter
-------------------------
['content_type', 'conversation_id', 'favorites', 'hashtags', 'mentions', 'recorded_time_seconds', 'retweet', 'tweetContent', 'tweet_id', 'userName']
hg-cpe
-------------------------
['_id', 'cpe.cpe_version', 'cpe.edition', 'cpe.language', 'cpe.other', 'cpe.part', 'cpe.product', 'cpe.software_edition', 'cpe.target_hardware', 'cpe.target_software', 'cpe.type', 'cpe.update', 'cpe.vendor', 'cpe.version', 'cpe_23_name', 'item_name', 'language', 'parsed', 'title']
hg-cve
-------------------------
['_id', 'cpe.cpe_version', 'cpe.edition', 'cpe.language', 'cpe.other', 'cpe.part', 'cpe.product', 'cpe.software_edition', 'cpe.target_hardware', 'cpe.target_software', 'cpe.type', 'cpe.update', 'cpe.vendor', 'cpe.version', 'cpe_23_name', 'item_name', 'language', 'parsed', 'title']
hg-blogs
-------------------------
['_id', 'html', 'link', 'parsed', 'published', 'site', 'text', 'title']
asu-hacking-items
-------------------------
['autoTranslated', 'entity', 'fi